# Pandas Project

*Este repositorio es un proyecto sobre la librería `pandas`de python que he realizado como tarea durante el bootcamp de Data Analytics de [Ironhack](https://www.ironhack.com/). Se trata del primer proyecto que realizamos.

Para realizarlo, hemos recibido el dataset *Global Shark Attack Incidents* disponible en [kaagle.com](https://www.kaggle.com/teajay/global-shark-attacks/version/1/kernels). Se trata de un archivo csv con datos sobre ataques de tiburones alrededor de todo el mundo.

El objetivo del proyecto es plantear una hipótesis y confirmarla con los datos del dataset. Sin embargo, los datos de la bbdd son muy caóticos, por lo que es necesario filtrar y limpiar los datos para poder confirmarla.

## Cargando el dataset

In [1]:
import pandas as pd
import re

df = pd.read_csv("./GSAF5.csv", encoding="latin1")
display(df.head())

print(df.columns)
print(df.shape)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')
(5992, 24)


La base de datos contiene 5992 valores y 24 columnas. No obstante, muchos de las columnas contienen información poco útil (como las dos últimas, 'Unnamed: 22', 'Unnamed: 23') y la tabla contiene muchos valores nulos en algunas columnas.

In [2]:
null_cols = df.isnull().sum()
print(null_cols)

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64


## Mi hipótesis

La hipótesis que quiero comprobar es:

    Los tiburones pequeños son más peligrosos

La base de datos no tiene una columna de tamaño. Sin embargo, la columna `Species` contiene en muchas filas información sobre el tamaño del tiburón que ha mordido.

La columna `Species` contiene muchos valores nulos y los no nulos a lo mejor no contienen datos sobre el tamaño. Por tanto, los resultados me darán una muestra que a lo mejor no es representativa de la población. Sin embargo, se trata de un ejercicio de clase y he elegido dicha hipótesis porque:

- Me parece un ejercicio interesante de regex
- Contiene datos numéricos, al que le puedo aplicar más fácilmente las funciones estadísticas que hemos visto en clase.


## Limpieza del dataset

Crearé una nueva columna `Sizes` donde guardaré los tamaños. Compararé el número de ataques y su mortalidad con los tamaños. Por lo tanto, la limpieza es sencilla: Para verificar mi hipótesis únicamente necesito las columnas tamaño y `Fatal (Y/N)`.

Para empezar, consultado el csv se observa que las columnas `Species` y `Sex` tienen un espacio después de la palabra. Normalizamos las columnas:

In [3]:
df.rename(index=str, columns={"Sex ": "Sex", "Species ": "Species"}, inplace = True)

Por otra parte, en la columna `Type` hay columnas con el valor *Invalid*. Consultando varios PDFs adjuntos en la columna `href`, se observa que dichos ataques son avisos falsos, mordeduras post-mortem o casos muy muy dudosos.

Por tanto, decido borrar todos dichos datos.

In [4]:
df=df[df["Type"] != "Invalid"].copy()

In [5]:
null_cols = df.isnull().sum()
print(null_cols)
print(df.shape)

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     38
Area                       358
Location                   445
Activity                   407
Name                       147
Sex                        494
Age                       2383
Injury                      20
Fatal (Y/N)                  5
Time                      2837
Species                   2655
Investigator or Source      12
pdf                          0
href formula                 1
href                         2
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5472
Unnamed: 23               5472
dtype: int64
(5473, 24)


### Limpieza de la columna `Species`

Por otra parte, observo que hay algunos datos en la columna `Species` que recogen, en una sola fila, tiburones de varios tamaños. Como por ejemplo, las siguientes:

In [6]:
aux = df[(df["Species"].str.contains("[\d\.]+\sm\s&\s[\d\.]+\sm|[\d\.]+\s'\s&\s[\d\.]+\s'")==True)]
print(aux.shape)
aux["Species"].value_counts()

(5, 24)


Two sharks seen in vicinity: 2.4 m & 4.25 m  [8' & 14'] TL    1
Said to involve 2 sharks: 5.2 m & 6 m [17' & 20']             1
4.5 m & 5 m white shark                                       1
2 scalloped hammerhead sharks, 1.5 m & 1.8 m [5' & 6']        1
2 sharks, 4.5 m & 3 m                                         1
Name: Species, dtype: int64

Para poder contabilizar dichos ataques como 2, en vez de como uno solo, decido hacer los siguiente:

* Añado dos filas al final de mi dataframe que son copias de la fila con dos ataques.
* Les cambio el valor de la fila `Species` por los dos tamaños diferentes.
* Borro de mi dataset la fila con el valor duplicado.

Esto lo he hecho con una función al que le enviamos el dataframe a modificar, la columna afectada y una expresión regular que saca por grupos los valores que queremos duplicar.

In [7]:
def quitarDuplicados(fdf, col, regex):
    aux_fdf = fdf.copy()
    for e in fdf[col].fillna("UNKNOWN"):
        if re.search(regex, e):
            r = re.findall(regex, e)
            for i in r[0]:
                aux_fdf = pd.concat([aux_fdf.copy(), aux_fdf[aux_fdf[col]==e].copy()], ignore_index = True)
                aux_fdf[col][aux_fdf.index.max()] = i
            aux_fdf = aux_fdf[aux_fdf[col]!=e].copy()
    return aux_fdf.copy()
        
        
print(df.shape)    
df = quitarDuplicados(df.copy(), "Species", "([\d\.]+')\s&\s([\d\.]+')").copy()
print(df.shape)
df = quitarDuplicados(df.copy(), "Species", "([\d\.]+\sm)\s&\s([\d\.]+\sm)").copy()
print(df.shape)
display(df.iloc[-5:])

(5473, 24)
(5476, 24)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(5478, 24)


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5474,1959.01.17.b,17-Jan-59,1959,Unprovoked,AUSTRALIA,Tasmania,Safety Cove,In deep water about 100 yards from his ship,"Brian Derry, a Naval Rating",M,...,20',"Odessa American, 1/19/1959; G.P. Whitley, ref ...",1959.01.17.b-Derry.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1959.01.17.b,1959.01.17.b,2105,NaN,NaN
5475,2004.12.16,16-Dec-04,2004,Unprovoked,AUSTRALIA,South Australia,"West Beach, Adelaide",Scurfing (surfboard being towed behind a boat),Nick Peterson,M,...,4.5 m,"P. Kemp & T. Peake, GSAF",2004.12.16-Peterson-draft.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2004.12.16,2004.12.16,4598,NaN,NaN
5476,2004.12.16,16-Dec-04,2004,Unprovoked,AUSTRALIA,South Australia,"West Beach, Adelaide",Scurfing (surfboard being towed behind a boat),Nick Peterson,M,...,5 m,"P. Kemp & T. Peake, GSAF",2004.12.16-Peterson-draft.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2004.12.16,2004.12.16,4598,NaN,NaN
5477,2004.07.10,10-Jul-04,2004,Unprovoked,AUSTRALIA,Western Australia,"Lefthanders Beach, Margaret River",Surfing,Bradley Adrian Smith,M,...,4.5 m,"T. Peake, GSAF",2004.07.10-BradSmith.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2004.07.10,2004.07.10,4560,NaN,NaN
5478,2004.07.10,10-Jul-04,2004,Unprovoked,AUSTRALIA,Western Australia,"Lefthanders Beach, Margaret River",Surfing,Bradley Adrian Smith,M,...,3 m,"T. Peake, GSAF",2004.07.10-BradSmith.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2004.07.10,2004.07.10,4560,NaN,NaN


Como se observa, antes de aplicar la función, el dataframe tenía 5473 filas y después 5478. Es decir hemos agregado (*10 filas nuevas - 5 filas eliminadas*) **5 filas nuevas**.

Las últimas filas mostradas están duplicadas a excepción de los tamaños, que son los que estaban recogidos anteriormente, es decir:

<pre>
Two sharks seen in vicinity: 2.4 m & 4.25 m  [8' & 14'] TL
Said to involve 2 sharks: 5.2 m & 6 m [17' & 20']
<b style="color:red;">4.5 m</b> & <b style="color:red;">5 m</b> white shark
2 scalloped hammerhead sharks, 1.5 m & 1.8 m [5' & 6']
2 sharks, <b style="color:red;">4.5 m</b> & <b style="color:red;">3 m</b>
</pre>

A continuación, limpiamos la columna `Species` para quedarnos solo con los tamaños. Para ello, creo una nueva columna que llamaremos `Sizes` donde guardaremos los datos limpios.

La columna la limpieza a través de una función y mediante *regex*.

In [ ]:
A continuación, limpiamos